In [2]:
!wget -P "./data/" "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet"

--2024-07-27 07:14:46--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 3.160.203.53, 3.160.203.81, 3.160.203.184, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|3.160.203.53|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38139949 (36M) [application/x-www-form-urlencoded]
Saving to: ‘yellow_tripdata_2022-01.parquet’

yellow_tripdata_202 100%[===================>]  36.37M  17.7MB/s    in 2.1s    

2024-07-27 07:14:49 (17.7 MB/s) - ‘yellow_tripdata_2022-01.parquet’ saved [38139949/38139949]



In [1]:
import pandas as pd